Script untuk melakukan testing dari model CNN terhadap Dataset


**Metrik yang didapatkan:**
*   Akurasi Keseluruhan
*   Confusion Matrix
*   Precision, Recall, dan F1-Score


**Prasyarat:**
*   Memiliki model hasil training CNN ( format .h5)
*   Memiliki dataset testing (format .zip)
*   Kedua file tersebut sudah diupload ke Google Drive


**Cara menggunakan:**
1. Upload script kode ini ke Google Drive, lalu buka menggunakan Google Colab
2. Upload file model dan data testing ke direktori Google Drive yang diinginkan
3. Ubah variabel `model_path` sesuai lokasi model
4. Ubah variabel `gdrive_zip` sesuai lokasi file zip data test
5. Sesuaikan nama folder dataset pada variabel `base_dir` dan `test_dir`
6. Ubah parameter input size `img_height` `img_width` sesuai dengan model training
7. Jalankan cell satu persatu













In [ ]:
# menghububgkan aplikasi ke Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# memuat model
import tensorflow as tf
from tensorflow import keras

# ATUR DIREKTORI MODEL
model_path = '/content/drive/MyDrive/Landsat/Model/ResNet50 x224.h5'
model = keras.models.load_model(model_path)

In [ ]:
# memuat dataset
import zipfile,os,shutil

# ATUR DIREKTORI DATASET
gdrive_zip = '/content/drive/MyDrive/Landsat/Dataset/EuroSAT 6-2-2.zip'
zip_ref = zipfile.ZipFile(gdrive_zip, 'r') # opens the zip file in read mode
zip_ref.extractall('/tmp') # ekstrak file ke folder /tmp (direktori pada Google Colab)
zip_ref.close()

# ATUR DIREKTORI FOLDER DATA UNTUK TESTING SESUAI NAMA FOLDER
base_dir = '/tmp/EuroSAT 6-2-2'
test_dir = os.path.join(base_dir, 'test')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# SAMAKAN INPUT SIZE MODEL TRAINING
img_width = 224
img_height = img_width 

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=1,
    class_mode='categorical',
    shuffle=False)

In [ ]:
# menghitung akurasi 
result = model.evaluate(test_generator)

In [ ]:
# menghitung probabilitas
y_predict = model.predict(test_generator)

In [ ]:
import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report
import numpy as np
import matplotlib.pyplot as plt

true_label = test_generator.classes # mendapatkan label sebenarnya
predicted_label = np.argmax(y_predict, axis=1) # mendapatkan label hasil klasifikasi

In [ ]:
# menghitung confusion matrix
cm = confusion_matrix(true_label, predicted_label)
# print(cm)

classes = ['anc', 'frs', 'hvg', 'hgw', 'ids', 'pst', 'pmc', 'rsd', 'rvr', 'slk']
classes_idx = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

# menampilkan confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
disp.plot()
plt.show()

In [ ]:
# menampilkan metrik untuk setiap kelas
metrics_report = classification_report(true_label, predicted_label)
# parameter output_dict=True
print(metrics_report)
